In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Run

In [1]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.runner as runner
import source.util as util

import source.util as util
import numpy as np
import pandas as pd
from importlib import *
import source.util as util
from copy import deepcopy, copy
from contextlib import redirect_stdout
import time 
import os
import re
import functools
import concurrent.futures

In [182]:
reload(util)

<module 'source.util' from '/home/lorenzo/Scrivania/Polimi/Thesis/code/onlineSG/source/util.py'>

In [10]:
experiment_folder = "../NewExperiments/"

In [138]:
def get_el(d, lst):
    if not lst:
        return d
    else:
        return get_el(d[lst[0]], lst[1:])

## parse batches

In [177]:
batches = []
bdict = {}
csv_pattern = re.compile("^.+\.csv$")
for root, dirs, files in os.walk(experiment_folder):
    if root != experiment_folder:
        args = root.split(experiment_folder)[-1].split("/")
        if not files:
            get_el(bdict, args[:-1])[args[-1]] = {}
        else:
            get_el(bdict, args[:-1])[args[-1]] = []
            for f in files:
                if (csv_pattern.match(f) and f != "batch.csv" and
                    f != "stats.csv" and f.split('.csv')[0] not in dirs):
                    #print(f)
                    b = runner.Batch(os.path.join(root, f), root)
                    b.parse_batch()
                    batches.append(b)
                    get_el(bdict, args).append(b)

In [178]:
len(batches)

20

## run batches in parallel

In [179]:
#with open('u_status.txt', 'w') as f:
#    with redirect_stdout(f):
with concurrent.futures.ProcessPoolExecutor(workers) as executor:
    futures = {}
    for b in batches:
        b.run(futures, executor, n=3)

## collect results

In [180]:
for b in batches:
    b.collect(futures)
    confs = b.configurations
    # util.plot_conf2("exp_regret", c, b.results_folder_path)
    dlist = [{"name": c.game.players[0].__class__.name,
              "avgs": c.stats["exp_regret"], 
              "lb": c.stats["lb_exp_regret"],
              "ub": c.stats["ub_exp_regret"]}
             for c in confs]
    util.plot_dicts(dlist, save=True, show=False, semilog=True,
                    path=b.results_folder_path,
                    title=confs[0].game.players[1].__class__.name)

0
1
2
0
1
2
0
1
2
0
1
2
0
1
2


/home/lorenzo/miniconda2/envs/python3/lib/python3.5/site-packages/matplotlib/ticker.py:2041: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  "Data has no positive values, and therefore cannot be "


0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2


## plot

In [183]:
for t in bdict:
    for p in bdict[t]:
        confs = [c for i in bdict[t][p]
                   for b in bdict[t][p][i]
                   for c in b.configurations]
        cdict = {}
        for c in confs:
            cname = c.name.split(" vs")[0]
            if cname not in cdict:
                cdict[cname] = [c]
            else:
                cdict[cname].append(c)
        dlist = []
        for n in cdict:
            avg_regrets = [c.stats["exp_regret"] for c in cdict[n]]
            d = util.avg_with_conf(avg_regrets, name=n)
            dlist.append(d)
        util.plot_dicts(dlist, save=True, show=False, semilog=True,
                path=os.path.join(experiment_folder, t, p),
                title=p)

## Experiments

verify what happened if something went wrong

In [47]:
e = runner.Experiment(g)

In [49]:
e.run_interaction()
print(e.game.strategy_history[-1])
#print(util.game_str(e.game, lenght=7))
print(e.game.history[-1])

KeyError: <FixedActionDefender id:0 resources:1>